This work is inspicred by Eric Heitz [Understanding the Masking-Shadowing Function in Microfacet-Based BRDFs](http://jcgt.org/published/0003/02/03/), check it out if you hadn't.

A BRDF derived from microfacet theory has the form $\frac {F(w_i, w_h, \eta) D(w_h) G(w_i, w_o, w_h)} {4 \overline\cos\theta_i \overline\cos\theta_o}$, sometimes $\frac {1} {4 \overline\cos\theta_i \overline\cos\theta_o}$ is been cancelled by G term so that it has the form $F(w_i, w_h, \eta) D(w_h) G(w_i, w_o, w_h)$:
* F: [fresnel equation](fresnel_equation.ipynb).
* D: normal distributuion function (NDF in abbrev.).
* G: masking and shadowing function, it's a composition of two visibility function (named $G_1$ by convention) for view and light direction.

A physically based BRDF must satify following constrains:
* The (signed) projected area of the microsurface is the same as the projected area of the macrosurface for any direction $v$
$$\int_{\Omega} D(m) \overline{(v \cdot m)} d{W_m} = v \cdot n$$
and in the special case $v=n$
$$\int_{\Omega} D(m) \overline{(n \cdot m)} d{W_m} = 1$$

* The projected area of the geometric surface onto the outgoing direction is also the projected area of the visible microsurface
$$\int_{\Omega} G_1(o, m) \overline{(o \cdot m)} D(m) d{W_m} = \overline{o \cdot n}$$

* If we don't take into accout the fresnel and shadowing, remain part of the BRDF $\frac {G_1(o, m) D(m)} {4 \cos \theta_i \cos \theta_o}$ can be treat as the light reflect from visible normal surface, since in this model, no energy was lost so that the integral should be 1:
$$\int_{\Omega} \frac {G_1(o, m) D(m)} {4 \cos \theta_o} d(W_i) = 1$$

In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [27]:
def lerp(t, v1, v2):
    return (1 - t) * v1 + t * v2;

def sqr(x):
    return x * x

def saturate(x):
    return np.clip(x, 0.0, 1.0)

def sphericalToCoordinates(theta, phi):
    return np.array([np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)])

def roughnessToAlpha(roughness):
    return roughness * roughness

def hemisphere_integrate(func):
#     def integrand(phi, theta):
#         w = sphericalToCoordinates(theta, phi)
#         return func(w) * np.sin(theta)
#     return integrate.dblquad(integrand, 0, np.pi/2-0.00001, lambda _: 0, lambda _: np.pi*2)
    dtheta = 0.05/4
    dphi = 0.05
    integral = 0.0
    for theta in np.arange(0, np.pi/2, dtheta):
        for phi in np.arange(0, np.pi*2, dphi):
            w = sphericalToCoordinates(theta, phi)
            integral += func(w) * np.sin(theta)
    return integral * dtheta * dphi

# def spherical_integrand(theta, phi, func):
#     x = np.sin(theta) * np.cos(phi)
#     y = np.sin(theta) * np.sin(phi)
#     z = np.cos(theta)
#     return func([x, y, z]) * np.sin(theta)

# def NDF_integrand(theta, phi, ndf):
#     return spherical_integrand(theta, phi, lambda v: ndf(v) * np.cos(theta))



In [28]:
# http://graphicrants.blogspot.com/2013/08/specular-brdf-reference.html

def D_BlinnPhong(H, a):
    NdotH = H[2]
    a2 = a * a
    return 1 / (np.pi * a2) * np.power(NdotH, 2 / a2 - 2)

def D_Beckmann(H, a):
    cosTheta = H[2]
    cosTheta2 = cosTheta * cosTheta
    cosTheta4 = cosTheta2 * cosTheta2
    a2 = a * a
    return 1 / (np.pi * a2 * cosTheta4) * np.exp((cosTheta2 - 1) / (a2 * cosTheta2))

def D_GGX(H, a):
    cosTheta = H[2]
    cosTheta2 = cosTheta * cosTheta
    a2 = a * a
    d = cosTheta2 * (a2 - 1) + 1
    return a2 / (np.pi * d * d)

def D_GGX_Aniso(H, anisotropic, a):
    aspect = np.sqrt(1 - anisotropic * 0.9)
    ax = np.maximum(.001, a / aspect)
    ay = np.maximum(.001, a * aspect)

    NdotH = H[2]
    HdotX = H[0]
    HdotY = H[1]
    return 1 / (np.pi * ax*ay * sqr( sqr(HdotX/ax) + sqr(HdotY/ay) + NdotH*NdotH ))

In [29]:
def validate(NDF, roughness, anisotropic):
    alpha = roughnessToAlpha(roughness)
    
    def integrand(w):
        D = 0.0
        if NDF == "BlinnPhong":
            D = D_BlinnPhong(w, roughness)
        elif NDF == "Beckmann":
            D = D_Beckmann(w, roughness)
        elif NDF == "GGX":
            D = D_GGX(w, roughness)
        elif NDF == "GGX_Aniso":
            D = D_GGX_Aniso(w, anisotropic, roughness)
        cosTheta = w[2]
        return D * cosTheta

    result = hemisphere_integrate(integrand)
    print("Expected value = 1.0, numberically result = %s" % str(result))

NDF = widgets.Dropdown(options=['BlinnPhong', 'Beckmann', 'GGX', 'GGX_Aniso'], 
                       value='GGX',
                       description='Number:',
                       disabled=False)
anisotropic = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.6)
roughness = widgets.FloatSlider(min=0.1,max=1,step=0.1,value=0.6);
widgets.interact(validate, NDF=NDF, anisotropic=anisotropic, roughness=roughness)

interactive(children=(Dropdown(description='Number:', index=2, options=('BlinnPhong', 'Beckmann', 'GGX', 'GGX_…

<function __main__.validate(NDF, roughness, anisotropic)>

In [30]:
def G1_Beckman(V, alpha):
    theta_o = np.arccos(V[2])
    a = 1 / (alpha * np.tan(theta_o));
    Lambda = 0.0
    if a < 1.6:
        Lambda = (1 - 1.259*a + 0.396*a**2) / (3.535*a + 2.181*a**2);
    else:
        Lambda = 0.0
    G = 1.0 / (1 + Lambda);
    return G

def G1_GGX(V, a):
    a2 = a * a
    cosTheta = V[2]
    cosTheta2 = cosTheta * cosTheta
    return 2 * cosTheta / (cosTheta + np.sqrt(a2 + (1 - a2) * cosTheta2))

In [ ]:
def validate(thetaO, phiO, alpha):
    thetaO = thetaO * np.pi / 2 - 0.000001
    phiO = phiO * np.pi * 2
    Wo = sphericalToCoordinates(thetaO, phiO)

    G = G1_GGX(Wo, alpha)
    
    def integrand(Wh):
        return G * D_GGX(Wh, alpha) * saturate(np.dot(Wo, Wh))

    result = hemisphere_integrate(integrand)
    print("Expected value = {}, numberically result = {}".format(str(Wo[2]), str(result)))

theta = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.5);
phi = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.5);
alpha = widgets.FloatSlider(min=0.01,max=1,step=0.1,value=0.5);
widgets.interact(validate, thetaO=theta, phiO=phi, alpha=alpha)

In [33]:
def validate(thetaO, phiO, alpha):
    thetaO = thetaO * np.pi / 2 - 0.000001
    phiO = phiO * np.pi * 2
    Wo = sphericalToCoordinates(thetaO, phiO)

    G = G1_Beckman(Wo, alpha)

    def integrand(Wi):
        Wh = Wi + Wo
        Wh /= np.linalg.norm(Wh)
        cosThetaO = Wo[2]
        return G * D_Beckmann(Wh, alpha) / (4 * cosThetaO)

    result = hemisphere_integrate(integrand)
    print("Expected value = {}, numberically result = {}".format(str(1), str(result)))

theta = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.5);
phi = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.5);
alpha = widgets.FloatSlider(min=0.01,max=1,step=0.1,value=0.5);
widgets.interact(validate, thetaO=theta, phiO=phi, alpha=alpha)

interactive(children=(FloatSlider(value=0.5, description='thetaO', max=1.0), FloatSlider(value=0.5, descriptio…

<function __main__.validate(thetaO, phiO, alpha)>